## Arquitetura do Projeto

A estrutura de pastas deste projeto foi pensada para separar claramente as responsabilidades, seguindo um princípio fundamental da engenharia de software: a separação de interesses. Podemos dividir o projeto em quatro grandes áreas:

* **Configuração (`net.json`, `parameters.py`):** Define o "cenário" da nossa simulação. É aqui que dizemos *quais* componentes existem, *como* eles se conectam e quais são suas propriedades físicas.
* **Lógica (`qsn/app/`):** Define o "comportamento" dos nossos componentes. É o cérebro que dita como os nós agem e reagem uns aos outros.
* **Ferramentas (`qsn/utils/`):** Scripts de "suporte" que nos ajudam com tarefas auxiliares, como a gravação de logs.
* **Execução e Análise (`GUIA.ipynb`, `log.txt`):** Onde "interagimos" com a simulação e analisamos seus "resultados".

---

**A Pasta Principal: `qsn/`**
Esta é a pasta raiz do nosso código-fonte. Agrupar tudo aqui permite que o projeto seja tratado como um "pacote" Python, facilitando importações e a organização.

`qsn/app/`:

Aqui mora o "cérebro" da nossa simulação. Esta pasta contém a lógica que define como cada nó se comporta.

* `ghz_active/`: É um sub-pacote dedicado ao nosso protocolo. Dentro dele, encontramos os arquivos que definem o comportamento do Hub e dos Sensores:

    * `hub_ghz_active_app.py`: O Maestro do Protocolo. Esta aplicação é proativa e orquestra todo o processo.
        * Ação Principal: O método `start()` inicia tudo, enviando uma mensagem `PROPOSE_GHZ` para os sensores que deve monitorar.
        * Interação: Ao receber de volta uma mensagem `ACEPT_GHZ`, ele solicita formalmente o emaranhamento quântico com aquele sensor.
        * Decisão: Ao final do tempo estipulado, ele verifica com quantos sensores conseguiu se emaranhar. Se o número for suficiente, ele considera o protocolo um sucesso. Caso contrário, ele avisa os sensores que não responderam (`ATTEMPT_FAILED`) para que eles possam ativar seu plano de contingência.

    * `sensor_ghz_active_app.py`: O Participante Colaborativo (Plano A). Esta aplicação é reativa, esperando as instruções do Hub.
        * Ação Principal: Seu trabalho começa ao receber uma mensagem `PROPOSE_GHZ`. Em resposta, ele envia `ACEPT_GHZ`, confirmando sua participação.
        * Interação: Ele gerencia suas memórias e aguarda o resultado do emaranhamento.
        * Mecanismo de Falha: Sua função mais crítica é o método `fallback()`. Se receber uma mensagem `ATTEMPT_FAILED` do Hub, significa que o "Plano A" falhou. Ele então executa este método, que tem a incrível capacidade de "trocar" a si mesmo pela aplicação de fallback (`SensorGHZActiveFallBackApp`), ativando o "Plano B".

    * `sensor_ghz_active_fallback_app.py`: O Plano de Contingência (Plano B). Esta aplicação é simples, direta e só entra em cena quando o Plano A falha.
        * Ação Principal: Assim que é ativada, ela imediatamente executa seu método `fallback()`.
        * Interação: Sua única tarefa é realizar uma medição local para gerar um resultado clássico (um bit 0 ou 1) e enviá-lo para o Hub com a mensagem `CLASSICAL_FALLBACK`. Ela garante que, mesmo em caso de falha quântica, o Hub receba *alguma* informação.

    * `message_ghz_active.py`: O Dicionário de Comunicação. Este arquivo define o "idioma" que os nós usam para se comunicar.
        * `GHZMessageType`: É um enumerador que funciona como o "assunto" de um e-mail. Ele define todos os tipos de mensagens possíveis: `PROPOSE_GHZ`, `ATTEMPT_FAILED`, `CLASSICAL_FALLBACK`, etc.
        * `GHZMessage`: É o "envelope" que carrega a mensagem. Ele contém o tipo da mensagem e os dados necessários para cada uma (como o nome do hub, o resultado clássico, etc.).

`qsn/utils/`: 

Esta pasta contém scripts auxiliares que não fazem parte da lógica principal do protocolo, mas são essenciais para a execução e análise da simulação.
* `logging_setup.py`: Nosso "Diretor de Câmeras", responsável por configurar o que será gravado no arquivo de log.
* `tracked_modules.py`: A "lista VIP" de módulos que o logger deve observar no modo `custom`.

**Arquivos de Configuração na Raiz (`qsn/`):**
Estes arquivos definem o cenário da nossa simulação. Ao alterá-los, podemos testar diferentes topologias e condições de hardware sem mudar o código das aplicações.
* `net.json`: O "blueprint" da nossa rede. Descreve todos os nós (hubs, sensores) e como eles estão fisicamente conectados (canais quânticos e clássicos).
* `parameters.py`: O "painel de controle" da simulação. Contém o dicionário `CONFIG` que define os parâmetros de hardware (fidelidade, eficiência), os tempos da simulação e a relação lógica entre hubs e sensores.

**Arquivos no Diretório Principal do Projeto**
Estes são arquivos relacionados à execução, documentação e saída.
* `GUIA.ipynb`: Este notebook, que serve como um guia interativo e ambiente de testes.
* `log.txt`: O "diário de bordo" da simulação. Cada vez que o código é executado, este arquivo é reescrito com os eventos que configuramos para serem registrados.